# MEST DAY 3

## Morning Session

### Activation Functions
Recall that $\hat{y} = w.x$
* Sigmoid $\sigma(\hat{y}) = \frac{1}{1 + e^{-\hat{y}}} $
* Hyperbolic Tangent
* Rectified Linear Unit

### Playground
* Gradient Descent
* Learning Rates

* http://playground.tensorflow.org/

In [ ]:
!pip install tensorflow==2.0.0-beta0

In [1]:
import tensorflow as tf
import numpy as np

In [ ]:
print(tf.__version__)

In [ ]:
a = np.random.normal(0, 1, (5, 2))
b = np.random.normal(0, 1, (2, 3))

c = tf.matmul(a,b)
print(c)

In [ ]:
print(c.numpy())

In [ ]:
x = np.random.rand(10, 3)
w = np.random.randn(1, x.shape[1])
b = np.random.randn(x.shape[0], 1)

y_pred = tf.matmul(x, w, transpose_b=True) + b
print(y_pred)

### 0D Tensor

In [ ]:
print(tf.add(1,1))

### 1D Tensor

In [ ]:
_a = np.ones(5)
_a = tf.multiply(_a, 1)
print(_a)

In [ ]:
print(_a.shape)

### 2D Tensor

In [ ]:
_b = np.ones([2, 5])
_b = tf.multiply(_b, 1)
print(_b)
print(_b.shape)

### 3D Tensor

In [ ]:
from tensorflow import keras

file_path = keras.utils.get_file('cat.jpg', 'https://www.petmd.com/sites/default/files/what-does-it-mean-when-cat-wags-tail.jpg')
img_raw = tf.io.read_file(file_path)
print(repr(img_raw)[:100]+"...")

In [ ]:
img_tensor = tf.image.decode_image(img_raw)

print(img_tensor.shape)
print(img_tensor.dtype)

In [ ]:
print(img_tensor)

In [ ]:
img_final = tf.image.resize(img_tensor, [192, 192])
img_final = img_final/255.0
print(img_final.shape)
print(img_final.numpy().min())
print(img_final.numpy().max())

### Define a Linear Regression Model

$\hat{y} = w.x + b$

$L(y, \hat{y})$

In [11]:
class Model(object):
  def __init__(self):
    self.W = None
    self.b = None
    
  def __call__(self, x):
    if self.W == None:
      self.W = tf.Variable(tf.random.normal(shape=(1, x.shape[1])))
    if self.b == None:
      self.b = tf.Variable(tf.random.normal(shape=(x.shape[0], 1)))
    return tf.matmul(x, self.W, transpose_b=True) + self.b

In [12]:
model = Model()
output = model(tf.constant([3.0, 3.1, 1.9, 2.0, 2.5, 2.9], shape=(3,2)))
print(output)

tf.Tensor(
[[6.9633856]
 [3.3688776]
 [6.135957 ]], shape=(3, 1), dtype=float32)


### Define a Loss Function

In [4]:
@tf.function
def loss(y_pred, y):
  return tf.reduce_mean(tf.square(y-y_pred))

### Define a training function

In [5]:
def train(model, x, y, alpha):
  x = tf.convert_to_tensor(x, np.float32) 
  y = tf.convert_to_tensor(y, np.float32)
  with tf.GradientTape() as t:
    t.watch(x)
    current_loss = loss(model(x), y)
  #print(current_loss)
  dW, db = t.gradient(current_loss, [model.W, model.b])
  #print(dW, db)
  model.W.assign_sub(alpha * dW)
  model.b.assign_sub(alpha * db)

In [2]:
import pandas as pd

#from google.colab import drive
#drive.mount('/content/gdrive')

df = pd.read_csv('./boston/train.csv', index_col='ID')

In [3]:
train_df = df.sample(frac=0.8,random_state=0)
test_df = df.drop(train_df.index)

columns = ['nox', 'rm', 'chas', 'dis', 'ptratio', 'lstat', 'rad']

X_train = train_df[columns].values
X_test = test_df[columns].values
y_train = train_df[['medv']].values
y_test = test_df[['medv']].values

In [4]:
df[columns].head()

,nox,rm,chas,dis,ptratio,lstat,rad
ID,,,,,,,
1,0.538,6.575,0,4.0900,15.3,4.98,1
2,0.469,6.421,0,4.9671,17.8,9.14,2
4,0.458,6.998,0,6.0622,18.7,2.94,3
5,0.458,7.147,0,6.0622,18.7,5.33,3
7,0.524,6.012,0,5.5605,15.2,12.43,5


In [5]:
df[['medv']].head()

,medv
ID,
1,24.0
2,21.6
4,33.4
5,36.2
7,22.9


In [13]:
model = Model()
train(model, X_train, y_train, alpha=0.1)

### Logistic Regression

In [ ]:
class LogisticModel(object):
  def __init__(self):
    self.W = None
    self.b = None
    
  def __call__(self, x):
    if self.W == None:
      self.W = tf.Variable(tf.random.normal(shape=(1, x.shape[1])))
    if self.b == None:
      self.b = tf.Variable(tf.random.normal(shape=(x.shape[0], 1)))
    y = tf.matmul(x, self.W, transpose_b=True) + self.b
    return tf.math.sigmoid(y)

### Train for 10 epochs

In [14]:
print(model.W)

<tf.Variable 'Variable:0' shape=(1, 7) dtype=float32, numpy=
array([[ 1.0251641 , 23.1939    , -0.73740137, 14.599553  , 66.53072   ,
        39.247177  , 26.141218  ]], dtype=float32)>


In [15]:
epochs = 10
model = Model()
for i in range(epochs):
  train(model, X_train, y_train, alpha=0.1)

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [ ]:
print(model.W)

## Tensorflow with Keras API

In [6]:
import tensorflow as tf
from tensorflow import keras

In [8]:
# define a neural network
model = keras.Sequential([
    keras.layers.Dense(10, input_shape=(X_train.shape[1],), activation='relu'),
    keras.layers.Dense(1)
])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                80        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
# define a linear model with a regression output
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(X_train.shape[1],))
])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 8         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
adam = keras.optimizers.Adam(0.001)
model.compile(optimizer=adam, loss='mse')

In [10]:
model.fit(X_train, y_train, epochs=100, validation_split=0.1)

Train on 239 samples, validate on 27 samples
Epoch 1/100
239/239 [==============================] - 1s 2ms/sample - loss: 908.0616 - val_loss: 780.0603
Epoch 2/100
239/239 [==============================] - 0s 77us/sample - loss: 859.3132 - val_loss: 734.5468
Epoch 3/100
239/239 [==============================] - 0s 71us/sample - loss: 813.4574 - val_loss: 692.3656
Epoch 4/100
239/239 [==============================] - 0s 114us/sample - loss: 770.5230 - val_loss: 653.8230
Epoch 5/100
239/239 [==============================] - 0s 103us/sample - loss: 730.0578 - val_loss: 617.9487
Epoch 6/100
239/239 [==============================] - 0s 99us/sample - loss: 692.1219 - val_loss: 584.1835
Epoch 7/100
239/239 [==============================] - 0s 107us/sample - loss: 656.8638 - val_loss: 551.5739
Epoch 8/100
239/239 [==============================] - 0s 87us/sample - loss: 622.8188 - val_loss: 520.6196
Epoch 9/100
239/239 [==============================] - 0s 116us/sample - loss: 590.3013 -

239/239 [==============================] - 0s 94us/sample - loss: 106.0081 - val_loss: 91.4315
Epoch 76/100
239/239 [==============================] - 0s 102us/sample - loss: 104.8760 - val_loss: 90.5792
Epoch 77/100
239/239 [==============================] - 0s 88us/sample - loss: 103.6941 - val_loss: 89.7965
Epoch 78/100
239/239 [==============================] - 0s 77us/sample - loss: 102.6047 - val_loss: 89.1394
Epoch 79/100
239/239 [==============================] - 0s 86us/sample - loss: 101.5401 - val_loss: 88.4529
Epoch 80/100
239/239 [==============================] - 0s 91us/sample - loss: 100.3564 - val_loss: 87.6289
Epoch 81/100
239/239 [==============================] - 0s 74us/sample - loss: 99.3137 - val_loss: 86.8917
Epoch 82/100
239/239 [==============================] - 0s 83us/sample - loss: 98.2568 - val_loss: 86.1830
Epoch 83/100
239/239 [==============================] - 0s 76us/sample - loss: 97.2479 - val_loss: 85.4433
Epoch 84/100
239/239 [=====================

In [11]:
y_pred = model.predict(X_test)

In [12]:
comp = pd.DataFrame({'actuals': y_test.reshape(-1), 'predicted': y_pred.reshape(-1)})

In [13]:
comp.head(n=10)

,actuals,predicted
0,18.2,24.604994
1,24.7,23.737633
2,20.0,22.628204
3,19.7,23.315868
4,20.5,23.581490
5,23.3,26.026188
6,18.7,24.974365
7,33.0,27.869051
8,24.2,24.386507
9,20.8,22.616060


### Can we engineer new features?

In [52]:
# we need a new way of getting data into the model
def df_to_dataset(df, columns, shuffle=True, batch_size=64):
  df = df.copy()
  labels = df.pop('medv')
  features_df = df[columns]
  ds = tf.data.Dataset.from_tensor_slices( (dict(features_df), labels) )
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df))
  ds = ds.batch(batch_size)
  return ds

In [53]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df, test_size=0.1)
train_ds = df_to_dataset(train, columns)
val_ds = df_to_dataset(val, columns)

In [54]:
df[columns].describe()

,nox,rm,chas,dis,ptratio,lstat,rad
count,333.000000,333.000000,333.000000,333.000000,333.000000,333.000000,333.000000
mean,0.557144,6.265619,0.060060,3.709934,18.448048,12.515435,9.633634
std,0.114955,0.703952,0.237956,1.981123,2.151821,7.067781,8.742174
min,0.385000,3.561000,0.000000,1.129600,12.600000,1.730000,1.000000
25%,0.453000,5.884000,0.000000,2.122400,17.400000,7.180000,4.000000
50%,0.538000,6.202000,0.000000,3.092300,19.000000,10.970000,5.000000
75%,0.631000,6.595000,0.000000,5.116700,20.200000,16.420000,24.000000
max,0.871000,8.725000,1.000000,10.710300,21.200000,37.970000,24.000000


In [55]:
feature_columns = []

# numeric columns
for _col in columns:
  feature_columns.append(tf.feature_column.numeric_column(_col))
  
# bucketize number of rooms
rm_buckets = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('rm'), boundaries=[1, 2, 3, 4, 5, 6, 7, 8, 9])
rad_buckets = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('rad'), boundaries=[1, 5, 10])
nox_buckets = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('nox'), boundaries=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

feature_columns.append(rm_buckets)
feature_columns.append(rad_buckets)
feature_columns.append(nox_buckets)

# cross rooms and nox


In [65]:
featuresLayer = keras.layers.DenseFeatures(feature_columns)
model = keras.Sequential([
    featuresLayer,
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

model.fit(train_ds, epochs=200, validation_data=val_ds)

Epoch 1/200
5/5 [==============================] - 1s 131ms/step - loss: 506.5299 - val_loss: 0.0000e+00
Epoch 2/200
5/5 [==============================] - 0s 5ms/step - loss: 484.0879 - val_loss: 522.7888
Epoch 3/200
5/5 [==============================] - 0s 6ms/step - loss: 475.9933 - val_loss: 505.7102
Epoch 4/200
5/5 [==============================] - 0s 5ms/step - loss: 462.6585 - val_loss: 489.0371
Epoch 5/200
5/5 [==============================] - 0s 6ms/step - loss: 448.7806 - val_loss: 472.8332
Epoch 6/200
5/5 [==============================] - 0s 6ms/step - loss: 428.3048 - val_loss: 457.0757
Epoch 7/200
5/5 [==============================] - 0s 6ms/step - loss: 408.9639 - val_loss: 441.6085
Epoch 8/200
5/5 [==============================] - 0s 6ms/step - loss: 380.8148 - val_loss: 426.7402
Epoch 9/200
5/5 [==============================] - 0s 6ms/step - loss: 386.7602 - val_loss: 412.2796
Epoch 10/200
5/5 [==============================] - 0s 7ms/step - loss: 369.3383 - val_

5/5 [==============================] - 0s 5ms/step - loss: 45.9756 - val_loss: 39.2800
Epoch 163/200
5/5 [==============================] - 0s 6ms/step - loss: 46.9458 - val_loss: 39.0716
Epoch 164/200
5/5 [==============================] - 0s 5ms/step - loss: 47.2251 - val_loss: 38.8607
Epoch 165/200
5/5 [==============================] - 0s 5ms/step - loss: 43.4416 - val_loss: 38.7034
Epoch 166/200
5/5 [==============================] - 0s 5ms/step - loss: 40.6936 - val_loss: 38.4792
Epoch 167/200
5/5 [==============================] - 0s 6ms/step - loss: 44.4744 - val_loss: 38.2744
Epoch 168/200
5/5 [==============================] - 0s 6ms/step - loss: 42.4463 - val_loss: 38.0735
Epoch 169/200
5/5 [==============================] - 0s 5ms/step - loss: 46.5611 - val_loss: 37.8779
Epoch 170/200
5/5 [==============================] - 0s 5ms/step - loss: 41.7956 - val_loss: 37.6942
Epoch 171/200
5/5 [==============================] - 0s 5ms/step - loss: 46.0652 - val_loss: 37.5148
Epoc

In [66]:
y_preds = model.predict(val_ds)

In [67]:
_c = pd.DataFrame({'actuals': val['medv'], 'predicted':y_preds.reshape(-1)})

In [68]:
_c.head(n=10)

,actuals,predicted
ID,,
345,31.2,29.459932
226,50.0,12.835381
353,18.6,17.685524
15,18.2,14.355935
482,23.7,30.157345
478,12.0,36.572540
254,42.8,20.001629
154,19.4,12.120832
117,21.2,23.368683


$$

### TensorFlow Estimators

In [74]:
def train_fn():
  df = train_df.copy()
  labels = df.pop('medv')
  features_df = df[columns]
  ds = tf.data.Dataset.from_tensor_slices( (dict(features_df), labels) )
  ds = ds.shuffle(1000).batch(64).repeat(5)
  
  return ds

def val_fn():
  df = test_df.copy()
  labels = df.pop('medv')
  features_df = df[columns]
  ds = tf.data.Dataset.from_tensor_slices( (dict(features_df), labels) )
  ds = ds.batch(64).repeat(1)
  
  return ds

In [72]:
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/xm/rvswt1sx4rdf56_wpqz5rt4c0000gn/T/tmp71smebtj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a3c504b10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [76]:
estimator.train(input_fn=train_fn, steps=None)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
I

## Afternoon
### Pytorch 1.0
* You should probably restart the runtime at this point

In [ ]:
! pip install torch torchvision

In [ ]:
import torch

In [ ]:
a = torch.rand(2,2)
b = torch.rand(2,2)
c = a + b
print(c)

In [ ]:
d = torch.add(a, b)
print(d)

In [ ]:
# a += 5
print(a)
a.add_(5)
print(a)

In [ ]:
print(a.shape)

In [ ]:
print(a * b)
print(a)
a.mul(b)
print(a)
a.mul_(b)
print(a)

In [ ]:
a = torch.rand(10000, 10000)
b = torch.rand(10000, 10000)
a.matmul(b)

### Move to GPU

In [ ]:
print(torch.cuda.is_available())

In [ ]:
a = a.cuda()
b = b.cuda()
a.matmul(b)

In [ ]:
features = torch.randn( (1,5))
weights = torch.randn_like(features)
bias = torch.randn( (1,1))

print(torch.sum(features * weights + bias) )

In [ ]:
print(torch.mm(features, weights.view(5,1)) + bias)

### Gradients

In [ ]:
x = torch.autograd.Variable(torch.ones(2, 2), requires_grad=True)

In [ ]:
y = x.mean()
print(y)

In [ ]:
print(y.backward())

In [ ]:
print(x.grad)

In [ ]:
print(x.grad_fn)

In [ ]:
print(x.data)

In [ ]:
print(y.grad_fn)

### Import Data

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

df = pd.read_csv('/content/gdrive/My Drive/boston/train.csv', index_col='ID')

In [ ]:
train_df = df.sample(frac=0.8,random_state=0)
test_df = df.drop(train_df.index)

columns = ['nox', 'rm', 'chas', 'dis', 'ptratio', 'lstat', 'rad']

X_train = train_df[columns].values
X_test = test_df[columns].values
y_train = train_df[['medv']].values
y_test = test_df[['medv']].values

### Simple Linear Model

In [ ]:
class SimpleModel():
  def __init__(self, x, y, lr=0.001):
    self.X = torch.autograd.Variable(torch.from_numpy(x).type(torch.FloatTensor))
    self.y = torch.autograd.Variable(torch.from_numpy(y).type(torch.FloatTensor))
    self.W = torch.autograd.Variable(torch.randn(x.shape[1], 1), requires_grad=True)
    self.b = torch.autograd.Variable(torch.randn(1, 1), requires_grad=True)
    self.alpha = lr
    
  def pred(self):
    return torch.matmul(self.X, self.W) + self.b
  
  def loss_fn(self):
    loss = (self.y - self.pred()).pow(2).sum()/self.X.shape[0]
    for p in [self.W, self.b]:
      if not p.grad is None:
        p.grad.data.zero_()
    loss.backward()
    return loss.data.item()
  
  def optimize(self):
    self.W.data -= self.alpha * self.W.grad.data
    self.b.data -= self.alpha * self.b.grad.data
    
  def train(self, epochs=100):
    for i in range(epochs):
      l = self.loss_fn()
      if i%10 == 0:
        print('Step:{} -- Current Loss: {}'.format(i, l))
      self.optimize()

In [ ]:
model = SimpleModel(X_train, y_train)

In [ ]:
model.train(50)